In [5]:
import pandas as pd
from better_profanity import profanity
import swifter
import regex as re
from tqdm.notebook import tqdm

tqdm.pandas()

In [6]:
pd.set_option('display.max_colwidth', 1)


In [7]:
reddit_file = 'D://Year 2//Programming//DAC//External Project//csvs//reddit.csv'
effective_forum_file = 'D://Year 2//Programming//DAC//External Project//csvs//effectivealtruism_antidepressants.csv'
cancer_forum_file = 'D://Year 2//Programming//DAC//External Project//csvs//cancer_research.csv'
anti_depressant_file = 'D://Year 2//Programming//DAC//External Project//csvs//symptoms_and_withdrawals.csv'



In [54]:
reddit_df = pd.read_csv(reddit_file)
effective_forum_df = pd.read_csv(effective_forum_file)
cancer_forum_df = pd.read_csv(cancer_forum_file)
anti_depressant_df = pd.read_csv(anti_depressant_file)

In [9]:
anti_depressant_df.head(5)

Date  \
0  October 9, 2012   
1  March 8, 2013     
2  March 8, 2013     
3  April 10, 2013    
4  April 16, 2013    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [55]:
anti_depressant_df['year'] = anti_depressant_df['Date'].str.extract(r'(\d+)$')
anti_depressant_df = anti_depressant_df.query('year >= "2018" & Comment.notnull()')
anti_depressant_df['type'] = 'anti-depressants'

In [11]:
anti_depressant_df.head(5)

,Date,Comment,year,type
78,"January 4, 2018","bob, quite often we find people are sensitized to ""normal"" dosages; our recommendations for reinstatement are usually tiny amounts to start, to see what happens.\n On 12/20/2017 at 8:46 AM, Downbutnotout said:\nSo you only updose if you’re side effects are from getting the medication out of your system? So how do I know my effects are from the medication bring out of my system or just bring depressed? I added 3 mg of Effexor back in but how do I know that’s really the problem? My effects are anxiety, horrible depression. But how do I know it’s from the effexir 6 weeks ago, the upping of trazadone or the short reinstatement if prozac which has made me insane?How will I know the reinstatement of Effexor beads is going anything??\nExpand \n DBNO, we've discussed this in your Intro topic. Please go back and re-read it.",2018,anti-depressants
79,"January 15, 2018",I had several responses to my signature. I am very concerned about my well being. I am not feeling well and from your response I tapered too quickly. I want to work with my Dr. and be sure that I am doing the right thing. I added the 20 mg Prozac because I have nothing else and need to hear back from Psychiatrist. I tried to summarize my sign. To make it clearer.,2018,anti-depressants
80,"March 10, 2018","Unfortunately ,too late reinstatement throw me into kindling.It's been 6 years now .stranded whenin I reinstated Prozac after 10 months off .I got adverse reactions since that 2012 ,till now even if I switched to lexapro in 2016 nothing helped.",2018,anti-depressants
81,"April 10, 2018","So, I really never suffered from depression. I did suffer from anxiety. I had been taking around 1 mg of Xanax for approx 4 or 5 yrs. Due to life circumstances, I made a huge mistake. I was feeling overwhelmed and went to a psychiatrist for the first time in my life. I went on Lexapro, most of the time at 30mg for 2 yrs. I then lost my job. She added 2mg of Abilify for the last 8 mos. of those 2 yrs on Lexapro. Time passed and my life got better. I don’t believe the Lexapro or Abilify helped me in the least. I CT’d off the Lexapro and Abilify at the same time. After about a month, I went into a hell that is indescribable. I suffered for 5 mos and am amazed I am alive.\n I tried reinstating Lexapro using the liquid. It was a disaster. Someone on the forum gave me advice to stay away from the liquid. The person saved my life. The liquid was much too strong. I then got tablets. I split a 5mg tablet into quarters. I started taking 1.25mg. I stayed on that for a month. Very, very Slight improvement. I then took 2.5mg. Stayed at that for 4-6 weeks. Then 5 mg, 7.5, 10mg, 12.5, and finally 15mg. I stayed at each increase for 4-6 weeks hoping I could stop. 15mg is when the intense dread and fear subsided. Although there had been improvements along the way. At most every increase, I had a couple of days of awful adrenaline surges. It was rough. I was taking 1mg of Klonopin per day at the time. \n I am now tapering the Klonopin. I am down to .475mg per day. I haven’t returned to work. But I am alive. I am there for my children and husband. I feel that I am severely traumatized from what happened. \n I think many people would benefit from reinstating with a portion of a tablet rather than liquid. I would love to get off of the Lexapro someday if I can gather the courage and the strength to try. My heart goes out to anyone dealing with this. I was seriously planning to end my life, the suffering was so intense. No where near anything I had ever experienced in my entire life. \n Edited April 10, 2018 by ChessieCat\nadded spacing",2018,anti-depressants
82,"April 12, 2018","On 10/9/2012 at 7:17 AM, Altostrata said:\n There are reports of people with prolonged post-withdrawal syndrome who did better taking a drug at full dosage 2 years later. If you want to do this, please consult a doctor, we cannot advise you on it.\nHello, I'm wondering if there are any 

In [56]:
merged_df = pd.concat([anti_depressant_df[['Comment','type' ]]\
                       .rename(columns={'Comment' : 'comments'}),
                        reddit_df, effective_forum_df , cancer_forum_df ] , 
                       ignore_index=True)

In [13]:
merged_df.head(5)

,comments,type
0,"bob, quite often we find people are sensitized to ""normal"" dosages; our recommendations for reinstatement are usually tiny amounts to start, to see what happens.\n On 12/20/2017 at 8:46 AM, Downbutnotout said:\nSo you only updose if you’re side effects are from getting the medication out of your system? So how do I know my effects are from the medication bring out of my system or just bring depressed? I added 3 mg of Effexor back in but how do I know that’s really the problem? My effects are anxiety, horrible depression. But how do I know it’s from the effexir 6 weeks ago, the upping of trazadone or the short reinstatement if prozac which has made me insane?How will I know the reinstatement of Effexor beads is going anything??\nExpand \n DBNO, we've discussed this in your Intro topic. Please go back and re-read it.",anti-depressants
1,I had several responses to my signature. I am very concerned about my well being. I am not feeling well and from your response I tapered too quickly. I want to work with my Dr. and be sure that I am doing the right thing. I added the 20 mg Prozac because I have nothing else and need to hear back from Psychiatrist. I tried to summarize my sign. To make it clearer.,anti-depressants
2,"Unfortunately ,too late reinstatement throw me into kindling.It's been 6 years now .stranded whenin I reinstated Prozac after 10 months off .I got adverse reactions since that 2012 ,till now even if I switched to lexapro in 2016 nothing helped.",anti-depressants
3,"So, I really never suffered from depression. I did suffer from anxiety. I had been taking around 1 mg of Xanax for approx 4 or 5 yrs. Due to life circumstances, I made a huge mistake. I was feeling overwhelmed and went to a psychiatrist for the first time in my life. I went on Lexapro, most of the time at 30mg for 2 yrs. I then lost my job. She added 2mg of Abilify for the last 8 mos. of those 2 yrs on Lexapro. Time passed and my life got better. I don’t believe the Lexapro or Abilify helped me in the least. I CT’d off the Lexapro and Abilify at the same time. After about a month, I went into a hell that is indescribable. I suffered for 5 mos and am amazed I am alive.\n I tried reinstating Lexapro using the liquid. It was a disaster. Someone on the forum gave me advice to stay away from the liquid. The person saved my life. The liquid was much too strong. I then got tablets. I split a 5mg tablet into quarters. I started taking 1.25mg. I stayed on that for a month. Very, very Slight improvement. I then took 2.5mg. Stayed at that for 4-6 weeks. Then 5 mg, 7.5, 10mg, 12.5, and finally 15mg. I stayed at each increase for 4-6 weeks hoping I could stop. 15mg is when the intense dread and fear subsided. Although there had been improvements along the way. At most every increase, I had a couple of days of awful adrenaline surges. It was rough. I was taking 1mg of Klonopin per day at the time. \n I am now tapering the Klonopin. I am down to .475mg per day. I haven’t returned to work. But I am alive. I am there for my children and husband. I feel that I am severely traumatized from what happened. \n I think many people would benefit from reinstating with a portion of a tablet rather than liquid. I would love to get off of the Lexapro someday if I can gather the courage and the strength to try. My heart goes out to anyone dealing with this. I was seriously planning to end my life, the suffering was so intense. No where near anything I had ever experienced in my entire life. \n Edited April 10, 2018 by ChessieCat\nadded spacing",anti-depressants
4,"On 10/9/2012 at 7:17 AM, Altostrata said:\n There are reports of people with prolonged post-withdrawal syndrome who did better taking a drug at full dosage 2 years later. If you want to do this, please consult a doctor, we cannot advise you on it.\nHello, I'm wondering if there are any links to these reports? Trying to find people who reinstate late with success.",anti-depressants


In [57]:
merged_df.shape

(19172, 2)

## Pre-Processing data

In [58]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [59]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

In [60]:
def lemmatize_words(text):
    """custom function to lemmatize the words"""
    return " ".join([lemmatizer.lemmatize(word) for word in text.split(' ')])

In [49]:
merged_df.head()

,comments,type
0,"bob, quite often we find people are sensitized to ""normal"" dosages; our recommendations for reinstatement are usually tiny amounts to start, to see what happens.\n On 12/20/2017 at 8:46 AM, Downbutnotout said:\nSo you only updose if you’re side effects are from getting the medication out of your system? So how do I know my effects are from the medication bring out of my system or just bring depressed? I added 3 mg of Effexor back in but how do I know that’s really the problem? My effects are anxiety, horrible depression. But how do I know it’s from the effexir 6 weeks ago, the upping of trazadone or the short reinstatement if prozac which has made me insane?How will I know the reinstatement of Effexor beads is going anything??\nExpand \n DBNO, we've discussed this in your Intro topic. Please go back and re-read it.",anti-depressants
1,I had several responses to my signature. I am very concerned about my well being. I am not feeling well and from your response I tapered too quickly. I want to work with my Dr. and be sure that I am doing the right thing. I added the 20 mg Prozac because I have nothing else and need to hear back from Psychiatrist. I tried to summarize my sign. To make it clearer.,anti-depressants
2,"Unfortunately ,too late reinstatement throw me into kindling.It's been 6 years now .stranded whenin I reinstated Prozac after 10 months off .I got adverse reactions since that 2012 ,till now even if I switched to lexapro in 2016 nothing helped.",anti-depressants
3,"So, I really never suffered from depression. I did suffer from anxiety. I had been taking around 1 mg of Xanax for approx 4 or 5 yrs. Due to life circumstances, I made a huge mistake. I was feeling overwhelmed and went to a psychiatrist for the first time in my life. I went on Lexapro, most of the time at 30mg for 2 yrs. I then lost my job. She added 2mg of Abilify for the last 8 mos. of those 2 yrs on Lexapro. Time passed and my life got better. I don’t believe the Lexapro or Abilify helped me in the least. I CT’d off the Lexapro and Abilify at the same time. After about a month, I went into a hell that is indescribable. I suffered for 5 mos and am amazed I am alive.\n I tried reinstating Lexapro using the liquid. It was a disaster. Someone on the forum gave me advice to stay away from the liquid. The person saved my life. The liquid was much too strong. I then got tablets. I split a 5mg tablet into quarters. I started taking 1.25mg. I stayed on that for a month. Very, very Slight improvement. I then took 2.5mg. Stayed at that for 4-6 weeks. Then 5 mg, 7.5, 10mg, 12.5, and finally 15mg. I stayed at each increase for 4-6 weeks hoping I could stop. 15mg is when the intense dread and fear subsided. Although there had been improvements along the way. At most every increase, I had a couple of days of awful adrenaline surges. It was rough. I was taking 1mg of Klonopin per day at the time. \n I am now tapering the Klonopin. I am down to .475mg per day. I haven’t returned to work. But I am alive. I am there for my children and husband. I feel that I am severely traumatized from what happened. \n I think many people would benefit from reinstating with a portion of a tablet rather than liquid. I would love to get off of the Lexapro someday if I can gather the courage and the strength to try. My heart goes out to anyone dealing with this. I was seriously planning to end my life, the suffering was so intense. No where near anything I had ever experienced in my entire life. \n Edited April 10, 2018 by ChessieCat\nadded spacing",anti-depressants
4,"On 10/9/2012 at 7:17 AM, Altostrata said:\n There are reports of people with prolonged post-withdrawal syndrome who did better taking a drug at full dosage 2 years later. If you want to do this, please consult a doctor, we cannot advise you on it.\nHello, I'm wondering if there are any links to these reports? Trying to find people who reinstate late with success.",anti-depressants


In [62]:
def pre_process(dataframe):

    #converts text to lowercase
    dataframe['comments'] = dataframe['comments'].apply(lambda x: x.lower())
    #Removes any other whitespace 
    dataframe['comments'] = dataframe['comments'].str.replace(r'\n|\t|\\w|\s ', '  ')
    
    #Remove usernames  of certain comments
    dataframe['comments'] = dataframe['comments'].str.replace(r"^.+\d+.+\w+\s*said\W", "")

    #remove numbers
    dataframe['comments'] = dataframe['comments'].str.replace(r"\d", "")

    #remove Hyperlinks
    dataframe['comments'] = dataframe['comments'].str.replace(r"<.*?>+|https?://\S+|www\.\S+", " ")
    
    #Remove all punctionations
    dataframe['comments'] = dataframe['comments'].str.replace(r"[^\w\s]_*", " ")

    #Removing Emojis and any other image related symbols
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoji
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)
    
    dataframe['comments'] = dataframe['comments'].apply(lambda x: emoji_pattern.sub(r' ', x))

    #Removing the stop words
   # dataframe['comments'] = dataframe['comments'].apply(remove_stopwords)


    #Lemmatize the words
    dataframe['comments'] = dataframe['comments'].apply(lemmatize_words) 

    #Removes cuss words parallely
   # dataframe['comments'] = dataframe['comments'].swifter.allow_dask_on_strings(enable=True).apply(profanity.censor)
    #dataframe['comments'] = dataframe['comments'].str.replace(r'\*\*\*\*', '')


    return dataframe

In [63]:
pre_process(merged_df)

,comments,type
0,so you only updose if you re side effect are from getting the medication out of your system so how do i know my effect are from the medication bring out of my system or just bring depressed i added mg of effexor back in but how do i know that s really the problem my effect are anxiety horrible depression but how do i know it s from the effexir week ago the upping of trazadone or the short reinstatement if prozac which ha made me insane how will i know the reinstatement of effexor bead is going anything expand dbno we ve discussed this in your intro topic please go back and re read it,anti-depressants
1,i had several response to my signature i am very concerned about my well being i am not feeling well and from your response i tapered too quickly i want to work with my dr and be sure that i am doing the right thing i added the mg prozac because i have nothing else and need to hear back from psychiatrist i tried to summarize my sign to make it clearer,anti-depressants
2,unfortunately too late reinstatement throw me into kindling it s been year now stranded whenin i reinstated prozac after month off i got adverse reaction since that till now even if i switched to lexapro in nothing helped,anti-depressants
3,so i really never suffered from depression i did suffer from anxiety i had been taking around mg of xanax for approx or yr due to life circumstance i made a huge mistake i wa feeling overwhelmed and went to a psychiatrist for the first time in my life i went on lexapro most of the time at mg for yr i then lost my job she added mg of abilify for the last mo of those yr on lexapro time passed and my life got better i don t believe the lexapro or abilify helped me in the least i ct d off the lexapro and abilify at the same time after about a month i went into a hell that is indescribable i suffered for mo and am amazed i am alive i tried reinstating lexapro using the liquid it wa a disaster someone on the forum gave me advice to stay away from the liquid the person saved my life the liquid wa much too strong i then got tablet i split a mg tablet into quarter i started taking mg i stayed on that for a month very very slight improvement i then took mg stayed at that for week then mg mg and finally mg i stayed at each increase for week hoping i could stop mg is when the intense dread and fear subsided although there had been improvement along the way at most every increase i had a couple of day of awful adrenaline surge it wa rough i wa taking mg of klonopin per day at the time i am now tapering the klonopin i am down to mg per day i haven t returned to work but i am alive i am there for my child and husband i feel that i am severely traumatized from what happened i think many people would benefit from reinstating with a portion of a tablet rather than liquid i would love to get off of the lexapro someday if i can gather the courage and the strength to try my heart go out to anyone dealing with this i wa seriously planning to end my life the suffering wa so intense no where near anything i had ever experienced in my entire life edited april by chessiecat added spacing,anti-depressants
4,there are report of people with prolonged post withdrawal syndrome who did better taking a drug at full dosage year later if you want to do this please consult a doctor we cannot advise you on it hello i m wondering if there are any link to these report trying to find people who reinstate late with success,anti-depressants
...,...,...
19167,i ha cycle of chemo the first were easier than the second when it changed to docetaxel i always felt ok for the first couple of day then rubbish for about a week i wa admitted to hospital in round and for high temp nothing found i just think that s how my body reacted to the chemo it s tough but you are stronger than you think you are just do what your body tell you to do and keep hydrated i found smoothy helped and haagen daaz ice cream a my taste bud were shocking i ve since had a mastectomy and

In [64]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [43]:
def is_english(text):
    doc = nlp(text)
    for token in doc:
        if token.lang_ != 'en':
            return False
    return True

In [29]:
words  = merged_df['comments']
english_words = list(filter(is_english, words))


In [31]:
merged_df['english_commetns']= english_words

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [66]:
tfidf = TfidfVectorizer()

In [67]:
X= tfidf.fit_transform(merged_df['comments'])

In [68]:
y = tfidf.get_feature_names_out()

In [69]:
print(y)

['_______________________' '_______________________________'
 '_________________________________________' ... 'جزيلا' 'شكرا' 'ἀσθένεια']
